In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

# Perform sqlite operations on disk
%env SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
%env TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
!echo $SQLITE_TMPDIR
!echo $TMPDIR
!echo $USERPROFILE

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-icelake/lib/python3.10/site-packages' at start of search paths.
env: SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
env: TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles



In [7]:
import pytorch_lightning 
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
from dataclasses import dataclass
import logging
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.TTE.task_heads.causal import TTETransformerForCausalSequenceModelling
from tqdm import tqdm
import time
import os
import polars as pl
pl.Config.set_tbl_rows(vocab_size + 1)
# TODO:
# replace experiment boilerplate with pytorch lightning

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# device = "cpu"    # if more informative debugging statements are needed
!pwd

cuda
/rds/homes/g/gaddcz/Projects/CPRD/examples/data


In [3]:
# Set config to be equivalent architecture of kaparthy benchmark, however they are not comparable tasks.
@dataclass
class DemoConfig:
    block_size: int = 256        # what is the maximum context length for predictions?
    n_layer: int = 6
    n_head: int = 6
    n_embd: int = 384
    bias: bool = True
    attention_type: str = "global"    
    dropout: float = 0.0
    unk_freq_threshold: float = 0.0
    TTELayer = "Exponential"

config = DemoConfig()

@dataclass
class OptConfig:
    batch_size: int = 128
    eval_interval: int = 1
    learning_rate: float = 3e-4
    epochs: int = 3
    
opt = OptConfig()

In [5]:
# Get a list of patients which fit a reduced set of criterion
# path_to_db = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/archive/Version2/"
path_to_db = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/"

# Build 
dm = FoundationalDataModule(path_to_db=path_to_db,
                            load=True,
                            tokenizer="tabular",
                            batch_size=opt.batch_size,
                            max_seq_length=config.block_size,
                            unk_freq_threshold=config.unk_freq_threshold,
                            min_workers=20,
                           )

vocab_size = dm.train_set.tokenizer.vocab_size
print(f"{vocab_size} vocab elements")

INFO:root:Loading Polars dataset from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/polars/
INFO:root:Using tokenizer tabular
INFO:root:Tokenzier created based on 3584.43M tokens
INFO:root:Creating split=train/ dataset
INFO:root:	 Loading split=train/ hash map for parquet
INFO:root:	 Hash map created for split=train/ with 22,912,046 samples
INFO:root:Creating split=test/ dataset
INFO:root:	 Loading split=test/ hash map for parquet
INFO:root:	 Hash map created for split=test/ with 1,207,449 samples
INFO:root:Creating split=val/ dataset
INFO:root:	 Loading split=val/ hash map for parquet
INFO:root:	 Hash map created for split=val/ with 1,226,576 samples


184 vocab elements


In [6]:
dm.train_set.view_sample(1, max_dynamic_events=12, report_time=True)

Time to retrieve sample index 1 was 0.1088871955871582 seconds

SEX                 | F
IMD                 | 4.0
ETHNICITY           | WHITE
birth_year          | 1997.0

Token                                                                      | Age               | Standardised value
O_E___height_1                                                             | 6639              | -0.03             
O_E___weight_2                                                             | 6639              | 0.19              
Systolic_blood_pressure_4                                                  | 6639              | -0.03             
Diastolic_blood_pressure_5                                                 | 6665              | 0.18              
Systolic_blood_pressure_4                                                  | 6665              | -0.03             
Body_mass_index_3                                                          | 7031              | 0.26              
Diastolic_blood_

In [8]:
print(dm.tokenizer._event_counts)

shape: (183, 3)
┌───────────────────────────────────┬───────────┬───────────┐
│ EVENT                             ┆ COUNT     ┆ FREQUENCY │
│ ---                               ┆ ---       ┆ ---       │
│ str                               ┆ u32       ┆ f64       │
╞═══════════════════════════════════╪═══════════╪═══════════╡
│ UNK                               ┆ 0         ┆ 0.0       │
│ ADDISONS_DISEASE                  ┆ 6691      ┆ 0.000002  │
│ CYSTICFIBROSIS                    ┆ 7053      ┆ 0.000002  │
│ SYSTEMIC_SCLEROSIS                ┆ 8772      ┆ 0.000002  │
│ SICKLE_CELL_DISEASE_V2            ┆ 11159     ┆ 0.000003  │
│ ADDISON_DISEASE                   ┆ 11794     ┆ 0.000003  │
│ DOWNSSYNDROME                     ┆ 17006     ┆ 0.000005  │
│ HAEMOCHROMATOSIS_V2               ┆ 18631     ┆ 0.000005  │
│ PLASMACELL_NEOPLASM_V2            ┆ 20301     ┆ 0.000006  │
│ SJOGRENSSYNDROME                  ┆ 23326     ┆ 0.000007  │
│ SYSTEMIC_LUPUS_ERYTHEMATOSUS      ┆ 26820     ┆ 0.00

In [9]:
meta_measurement = dm.train_set.meta_information["measurement_tables"]
# display(meta_measurement)

for measurement in dm.train_set.meta_information["measurement_tables"]["event"]:
    event_meta = meta_measurement[meta_measurement.event == measurement]
    display(event_meta.approx_lqr.to_numpy()[0])
    # print(event_meta[""])

-4.646246225231446

-36.374906271016215

3.4171338038240116

4.558863141370336

-4.327567643517323

...

0.5039839286086738

1.5954247277740436

-3.8672877345805388

-4.142799746985002

22.62504211170451

In [14]:
import pandas as pd
pd.set_option('display.max_rows', 1000) #replace n with the number of columns you want to see completely
for _key in dm.train_set.meta_information.keys():
    display(dm.train_set.meta_information[_key])


{'SEX':   category     count
 0        F  14278868
 1        I       683
 2        M  13841966,
 'IMD':    category    count
 0       NaN  2442913
 1       1.0  4789813
 2       2.0  4981882
 3       3.0  5072397
 4       4.0  5650727
 5       5.0  5183785,
 'ETHNICITY':   category     count
 0    ASIAN   2267997
 1    BLACK   1156866
 2  MISSING   8058247
 3    MIXED    485838
 4    OTHER    422374
 5    WHITE  15730195}

,event,count
0,ADDISONS_DISEASE,6691
1,ADDISON_DISEASE,11794
2,AF,731332
3,ALCOHOLMISUSE_V2,1125212
4,ALLCANCER_NOHAEM_NOBCC,1496973
5,ALLERGICRHINITISCONJ,3291165
6,ALL_DEMENTIA,528602
7,ANXIETY,3560978
8,ANY_DEAFNESS_HEARING_LOSS_V2,2282766
9,AORTICANEURYSM_V2,101134


,event,count,count_obs,digest,min,max,mean,approx_lqr,approx_uqr
0,25_Hydroxyvitamin_D2_level_92,782791,693470,"({'m': 0.0, 'c': 9.0}, {'m': 0.1, 'c': 112.0},...",0.000000e+00,6.860000e+02,3.908721e+00,-4.646246,10.806670
1,25_Hydroxyvitamin_D3_level_90,809104,781118,"({'m': 0.1, 'c': 3.0}, {'m': 1.0, 'c': 314.0},...",0.000000e+00,9.518000e+02,4.714889e+01,-36.374906,121.281425
2,AST___aspartate_transam_SGOT__46,1738489,1680613,"({'m': 0.0, 'c': 3901.0}, {'m': 0.770571428571...",0.000000e+00,1.533000e+04,2.661963e+01,3.417134,41.771075
3,AST_serum_level_47,10837982,10485351,"({'m': 0.0, 'c': 53.0}, {'m': 1.8, 'c': 1.0}, ...",-5.000000e+00,2.070000e+04,2.725168e+01,4.558863,41.966985
4,Albumin___creatinine_ratio_37,180911,78420,"({'m': -1.0, 'c': 1.0}, {'m': 0.0, 'c': 4213.0...",-1.000000e+00,1.282100e+04,1.067255e+01,-4.327568,8.831512
5,Basophil_count_22,86869779,85642540,"({'m': 0.0, 'c': 37098.0}, {'m': 0.01, 'c': 28...",-1.000000e-01,1.111110e+05,5.008992e-02,-0.093801,0.160919
6,Blood_calcium_level_38,415717,385464,"({'m': 0.0, 'c': 33.0}, {'m': 1.0, 'c': 1.0}, ...",0.000000e+00,4.400000e+02,2.352980e+00,2.025402,2.622520
7,Blood_urea_28,785766,671861,"({'m': 0.0, 'c': 2746.0}, {'m': 0.09, 'c': 1.0...",0.000000e+00,1.265000e+03,6.513018e+00,0.231978,11.019293
8,Body_mass_index_3,99868822,97759312,"({'m': 0.0, 'c': 14.0}, {'m': 0.05, 'c': 1.0},...",-3.268000e+04,2.100000e+09,2.933050e+02,10.469322,43.324813
9,Brain_natriuretic_peptide_level_66,229202,159318,"({'m': 0.0, 'c': 120.0}, {'m': 0.1, 'c': 1.0},...",0.000000e+00,5.001420e+05,4.168786e+02,-245.085949,483.166025
